# Agentic Retrieval Pipeline — Azure AI Search + Foundry Agent Service

This notebook creates an end-to-end agentic retrieval pipeline that integrates **Azure AI Search** with **Foundry Agent Service**.

Based on the [official tutorial](https://learn.microsoft.com/azure/search/agentic-retrieval-how-to-create-pipeline), with **added error handling and validation** at each step.

## Steps
1. Load connections
2. Create a search index
3. Upload documents to the index
4. Create a knowledge source
5. Create a knowledge base
6. Set up a project client
7. Create a project connection
8. Create an agent with the MCP tool
9. Chat with the agent
10. Clean up resources

## Prerequisites

Before running this notebook, make sure you have:

1. Completed `sample.env` → `.env` with your Azure resource values.
2. Run `az login` in your terminal.
3. Assigned **all** required RBAC roles (see README.md).
4. Enabled **system-assigned managed identity** on both Search and Foundry project.
5. Deployed `text-embedding-3-large` and `gpt-4.1-mini` (or your chosen models) in your Foundry project.
6. Verified that your AI Hub has associated ML resources in the Azure Portal → Resource Visualizer.

## Step 1 — Load connections

In [ ]:
import os
import sys

from azure.identity import DefaultAzureCredential
from azure.mgmt.core.tools import parse_resource_id
from dotenv import load_dotenv

load_dotenv(override=True)

# ── Required environment variables ──
required_vars = [
    "AZURE_SEARCH_ENDPOINT",
    "PROJECT_ENDPOINT",
    "PROJECT_RESOURCE_ID",
    "AZURE_OPENAI_ENDPOINT",
]

missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    sys.exit(f"ERROR: Missing environment variables: {', '.join(missing)}. "
             f"Copy sample.env to .env and fill in your values.")

project_endpoint = os.environ["PROJECT_ENDPOINT"]
project_resource_id = os.environ["PROJECT_RESOURCE_ID"]
project_connection_name = os.getenv("PROJECT_CONNECTION_NAME", "earthknowledgeconnection")
agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
agent_name = os.getenv("AGENT_NAME", "earth-knowledge-agent")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
knowledge_source_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_SOURCE_NAME", "earth-knowledge-source")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth-at-night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
base_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-knowledge-base")

# Parse the resource ID
parsed_resource_id = parse_resource_id(project_resource_id)
subscription_id = parsed_resource_id['subscription']
resource_group = parsed_resource_id['resource_group']
account_name = parsed_resource_id['name']
project_name = parsed_resource_id['child_name_1']

print("✅ Connections loaded successfully")
print(f"   Search endpoint : {endpoint}")
print(f"   Project endpoint: {project_endpoint}")
print(f"   OpenAI endpoint : {azure_openai_endpoint}")
print(f"   Agent model     : {agent_model}")
print(f"   Subscription    : {subscription_id}")
print(f"   Resource group  : {resource_group}")

## Step 2 — Create a search index

Creates an index with fields for text, embeddings, and semantic search.

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters,
    HnswAlgorithmConfiguration, SearchField, SearchIndex,
    SemanticConfiguration, SemanticField, SemanticPrioritizedFields,
    SemanticSearch, VectorSearch, VectorSearchProfile
)

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(
            name="page_embedding_text_3_large",
            type="Collection(Edm.Single)",
            stored=False,
            vector_search_dimensions=3072,
            vector_search_profile_name="hnsw_text_3_large",
        ),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True),
    ],
    vector_search=VectorSearch(
        profiles=[
            VectorSearchProfile(
                name="hnsw_text_3_large",
                algorithm_configuration_name="alg",
                vectorizer_name="azure_openai_text_3_large",
            )
        ],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model,
                ),
            )
        ],
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[SemanticField(field_name="page_chunk")]
                ),
            )
        ],
    ),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"✅ Index '{index_name}' created or updated successfully")

## Step 3 — Upload documents to the index

Loads NASA Earth-at-Night data from GitHub.

In [ ]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = (
    "https://raw.githubusercontent.com/Azure-Samples/"
    "azure-search-sample-data/refs/heads/main/"
    "nasa-e-book/earth-at-night-json/documents.json"
)

response = requests.get(url)
response.raise_for_status()
documents = response.json()
print(f"   Downloaded {len(documents)} documents")

with SearchIndexingBufferedSender(
    endpoint=endpoint, index_name=index_name, credential=credential
) as client:
    client.upload_documents(documents=documents)

print(f"✅ Documents uploaded to index '{index_name}'")

## Step 4 — Create a knowledge source

A reusable reference to the search index.

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndexFieldReference, SearchIndexKnowledgeSource,
    SearchIndexKnowledgeSourceParameters
)

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_fields=[
            SearchIndexFieldReference(name="id"),
            SearchIndexFieldReference(name="page_number"),
        ],
    ),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"✅ Knowledge source '{knowledge_source_name}' created or updated successfully")

## Step 5 — Create a knowledge base

Orchestrates agentic retrieval and exposes an MCP endpoint.

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    KnowledgeBase, KnowledgeRetrievalMinimalReasoningEffort,
    KnowledgeRetrievalOutputMode, KnowledgeSourceReference
)

knowledge_base = KnowledgeBase(
    name=base_name,
    knowledge_sources=[
        KnowledgeSourceReference(name=knowledge_source_name)
    ],
    output_mode=KnowledgeRetrievalOutputMode.EXTRACTIVE_DATA,
    retrieval_reasoning_effort=KnowledgeRetrievalMinimalReasoningEffort(),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_base(knowledge_base=knowledge_base)

mcp_endpoint = f"{endpoint}/knowledgebases/{base_name}/mcp?api-version=2025-11-01-Preview"

print(f"✅ Knowledge base '{base_name}' created or updated successfully")
print(f"   MCP endpoint: {mcp_endpoint}")

## Step 6 — Set up a project client

Connects to your Foundry project and lists existing agents.

In [ ]:
from azure.ai.projects import AIProjectClient

project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

existing_agents = list(project_client.agents.list())
print(f"✅ Connected to project. Found {len(existing_agents)} existing agent(s).")
for a in existing_agents:
    print(f"   - {a.name} (version {a.version})")

## Step 7 — Create a project connection

Creates an MCP connection in Foundry pointing to the knowledge base.

> **Common failure point**: If this step returns a 4xx error, verify:
> - Your `PROJECT_RESOURCE_ID` is correct.
> - You have the **Azure AI Project Manager** role.
> - The project's managed identity is enabled.

In [ ]:
import requests
from azure.identity import get_bearer_token_provider

bearer_token_provider = get_bearer_token_provider(
    credential, "https://management.azure.com/.default"
)

headers = {
    "Authorization": f"Bearer {bearer_token_provider()}",
}

connection_url = (
    f"https://management.azure.com{project_resource_id}"
    f"/connections/{project_connection_name}"
    f"?api-version=2025-10-01-preview"
)

connection_body = {
    "name": project_connection_name,
    "type": "Microsoft.MachineLearningServices/workspaces/connections",
    "properties": {
        "authType": "ProjectManagedIdentity",
        "category": "RemoteTool",
        "target": mcp_endpoint,
        "isSharedToAll": True,
        "audience": "https://search.azure.com/",
        "metadata": {"ApiType": "Azure"},
    },
}

response = requests.put(connection_url, headers=headers, json=connection_body)

if not response.ok:
    print(f"❌ Connection creation failed: {response.status_code}")
    print(f"   URL: {connection_url}")
    print(f"   Response: {response.text}")
    print("\n   Troubleshooting:")
    print("   - Verify PROJECT_RESOURCE_ID is correct in your .env file")
    print("   - Ensure you have 'Azure AI Project Manager' role on the project")
    print("   - Check that project managed identity is enabled")
    response.raise_for_status()
else:
    print(f"✅ Connection '{project_connection_name}' created or updated successfully")

## Step 8 — Create an agent with the MCP tool

> **Common failure point**: If the agent does not appear in Foundry:
> - Verify your model (`AGENT_MODEL`) is actually deployed in Models + Endpoints.
> - Ensure you have the **Azure AI User** role on the project.
> - Check that your AI Hub has ML resources (Resource Visualizer in Azure Portal).

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

instructions = """
You are a helpful assistant that must use the knowledge base to answer all the questions from user. You must never answer from your own knowledge under any circumstances.
Every answer must always provide annotations for using the MCP knowledge base tool and render them as: `\u3010message_idx:search_idx\u2020source_name\u3011`
If you cannot find the answer in the provided knowledge base you must respond with "I don't know".
"""

mcp_kb_tool = MCPTool(
    server_label="knowledge-base",
    server_url=mcp_endpoint,
    require_approval="never",
    allowed_tools=["knowledge_base_retrieve"],
    project_connection_id=project_connection_name,
)

try:
    agent = project_client.agents.create_version(
        agent_name=agent_name,
        definition=PromptAgentDefinition(
            model=agent_model,
            instructions=instructions,
            tools=[mcp_kb_tool],
        ),
    )
    print(f"✅ AI agent '{agent_name}' created successfully")
    print(f"   Agent name   : {agent.name}")
    print(f"   Agent version: {agent.version}")
except Exception as e:
    print(f"❌ Agent creation failed: {e}")
    print("\n   Troubleshooting:")
    print(f"   - Is '{agent_model}' deployed in your project's Models + Endpoints?")
    print("   - Do you have the 'Azure AI User' role on the project?")
    print("   - Does your AI Hub have ML resources? (Check Resource Visualizer)")
    raise

### Validate: Confirm the agent exists

In [ ]:
# Verify the agent was created
agents_after = list(project_client.agents.list())
agent_names = [a.name for a in agents_after]

if agent_name in agent_names:
    print(f"✅ Agent '{agent_name}' confirmed in agent list")
else:
    print(f"⚠️  Agent '{agent_name}' NOT found in agent list.")
    print(f"   Available agents: {agent_names}")
    print("   The chat step below will likely fail.")

## Step 9 — Chat with the agent

Uses the Conversations and Responses APIs to interact with the agent.

> **This is the step where BadRequestError 400 typically occurs.**
> The error handling below will print the full error details to help diagnose the issue.

In [ ]:
# Get the OpenAI client for responses and conversations
openai_client = project_client.get_openai_client()

try:
    conversation = openai_client.conversations.create()
    print(f"   Conversation created: {conversation.id}")
except Exception as e:
    print(f"❌ Failed to create conversation: {e}")
    print("\n   This usually means:")
    print("   - Your AI Hub is missing ML resources (most common)")
    print("   - The project endpoint is incorrect")
    print("   - 'Azure AI User' role is not assigned")
    raise

# Send the query to the agent
try:
    response = openai_client.responses.create(
        conversation=conversation.id,
        tool_choice="required",
        input=(
            "Why do suburban belts display larger December brightening "
            "than urban cores even though absolute light levels are higher downtown? "
            "Why is the Phoenix nighttime street grid so sharply visible from space, "
            "whereas large stretches of the interstate between midwestern cities "
            "remain comparatively dim?"
        ),
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )
    print(f"\n✅ Response received:\n")
    print(response.output_text)

except Exception as e:
    error_msg = str(e)
    print(f"❌ Chat request failed: {e}")
    print("\n" + "=" * 60)
    print("DIAGNOSTIC INFORMATION")
    print("=" * 60)
    print(f"Agent name    : {agent.name}")
    print(f"Agent version : {agent.version}")
    print(f"Agent model   : {agent_model}")
    print(f"MCP endpoint  : {mcp_endpoint}")
    print(f"Connection    : {project_connection_name}")
    print(f"Project endpt : {project_endpoint}")
    print()

    if "aml_connections_decode_error" in error_msg or "decode AML" in error_msg:
        print("🔍 ROOT CAUSE: AML connections decode error")
        print("   Your AI Hub is missing Machine Learning resources.")
        print("   FIX: Go to Azure Portal → Resource Group → Resource Visualizer")
        print("   and verify an ML Service exists alongside your AI Hub.")
        print("   If missing, recreate the project to provision ML resources.")
    elif "authentication" in error_msg.lower() or "unauthorized" in error_msg.lower():
        print("🔍 ROOT CAUSE: Authentication / authorization failure")
        print("   FIX: Run 'az login' again and verify all RBAC roles are assigned.")
        print("   Especially: 'Azure AI User' and 'Azure AI Project Manager'.")
    elif "not found" in error_msg.lower():
        print("🔍 ROOT CAUSE: Agent or model not found")
        print(f"   FIX: Verify '{agent_model}' is deployed in Models + Endpoints.")
        print(f"   Also verify agent '{agent.name}' exists (run validation cell above).")
    else:
        print("🔍 UNKNOWN ERROR — Full details above.")
        print("   Common fixes:")
        print("   1. Ensure AI Hub has ML resources (Resource Visualizer)")
        print("   2. Re-run 'az login'")
        print("   3. Verify all RBAC roles from README")
        print("   4. Ensure Search service is in a supported region")
        print("   5. Check that managed identities are enabled on both services")
    raise

## Step 10 — Inspect the response metadata

In [ ]:
# View the full response including citations and query metadata
try:
    response.to_dict()
except NameError:
    print("⚠️  No response to inspect — the chat step did not succeed.")

## Step 11 — Clean up resources (optional)

Run this cell to delete all objects created by this notebook.

In [ ]:
# ⚠️  Uncomment the lines below to delete resources

# # Delete the agent
# project_client.agents.delete_version(agent.name, agent.version)
# print(f"Deleted agent '{agent.name}' version '{agent.version}'")

# # Delete the knowledge base
# index_client.delete_knowledge_base(base_name)
# print(f"Deleted knowledge base '{base_name}'")

# # Delete the knowledge source
# index_client.delete_knowledge_source(knowledge_source=knowledge_source_name)
# print(f"Deleted knowledge source '{knowledge_source_name}'")

# # Delete the search index
# index_client.delete_index(index)
# print(f"Deleted index '{index_name}'")